# Adding Custom Functions to BALSA - Tutorial

This notebook demonstrates how to add your custom optimisation function to the BALSA benchmark suite.

## Prerequisites

First, let's ensure we have BALSA installed:

In [ ]:
%pip install balsa  # If not already installed

## 1. Creating Your Custom Function

Let's create a custom objective function by inheriting from `ObjectiveFunction`:

In [38]:
from dataclasses import dataclass, field
from typing import Any, override
import sys

sys.path.append("../../")  # Adds the project root directory to Python path

from balsa.obj_func import ObjectiveFunction
import numpy as np


@dataclass
class MyCustomFunction(ObjectiveFunction):
    """Custom objective function for optimisation.

    This is a simple example that implements a quadratic function.
    """

    name: str = "my_custom"
    dims: int = 5  # Example: 5-dimensional function
    turn: float = 0.1
    func_args: dict[str, Any] = field(
        default_factory=lambda: {
            "param1": 1.0,
            "param2": 2.0,
            "lb": [-5.0] * 5,  # Lower bounds
            "ub": [5.0] * 5,  # Upper bounds
        }
    )

    def __post_init__(self) -> None:
        super().__post_init__()
        assert self.dims > 0
        self.lb = np.array(self.func_args.get("lb"))
        self.ub = np.array(self.func_args.get("ub"))
        self.param1 = self.func_args.get("param1")
        self.param2 = self.func_args.get("param2")

    @override
    def _scaled(self, y: float) -> float:
        return y  # No scaling in this example

    @override
    def __call__(self, x: np.ndarray, saver: bool = True, return_scaled=False) -> float:
        self.counter += 1
        assert len(x) == self.dims
        assert x.ndim == 1

        # Example quadratic function
        y = float(np.sum(x**2))

        self.tracker.track(y, x, saver)
        return y if not return_scaled else self._scaled(y)

## 2. Testing the Function

Let's test our custom function with some sample inputs:

In [39]:
# Create an instance of our custom function
custom_func = MyCustomFunction()

# Test with a random input
test_input = np.random.uniform(low=-5.0, high=5.0, size=5)
result = custom_func(test_input)

print(f"Test input: {test_input}")
print(f"Function output: {result}")

Test input: [ 2.35529722 -1.21114182  4.05210127  3.35870916 -3.53762404]
Function output: 47.2295252332561


## 3. Using the Function with BALSA

Now let's set up and run an optimization using our custom function:

In [42]:
from balsa.active_learning import OptimisationRegistry, OptimisationConfig, ActiveLearningPipeline

# Register your custom function
OptimisationRegistry.FUNCTIONS["my_custom"] = MyCustomFunction

# Configuration
config = OptimisationConfig(
    dims=5,
    search_method="turbo",
    obj_func_name="my_custom",
    num_acquisitions=50,
    num_samples_per_acquisition=1,
    surrogate="default_surrogate",
    num_init_samples=30,
    func_args={
        "param1": 1.0,
        "param2": 2.0,
        "lb": [-5.0] * 5,
        "ub": [5.0] * 5
    }
)

# Run optimization
pipeline = ActiveLearningPipeline(config)
result = pipeline.run()

This optimisation is based on a turbo optimiser
Using dtype = torch.float32 
Using device = cpu
TR-0 starting from: 20.3
TR-1 starting from: 16.41
TR-2 starting from: 21.33
TR-3 starting from: 16.54
TR-4 starting from: 17.81
35) New best @ TR-0: 12.82
